In [1]:
import neuroglancer
import cloudvolume
import pandas as pd, os

## README
This example assumes you have precomputed datasets already made and hosted with cloudvolume.
To host the raw data in this example that I already converted to precomputed format, open an ipython terminal and do:
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/20170204_tp_bl6_cri_1750r_03/647')
vol.viewer(port=1337) # You can choose the port here, but remember it for later
```
The last command will cause the window to hang -- that is the expected behavior. Do not try to do this in this jupyter notebook as it will cause your session to hang and you won't be able to run any of the following steps.

## Set the neuroglancer client you want to use. 
Default is Seung lab's which should work fine for this example

In [2]:
neuroglancer.set_static_content_source(url='https://neuromancer-seung-import.appspot.com')

# Load in the raw data into neuroglancer and generate the link

In [3]:
# This volume handle can be used to notify the viewer that the data has changed.
viewer = neuroglancer.Viewer()

# Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume
with viewer.txn() as s:
    s.layers['20170204_tp_bl6_cri_1750r_03'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337',
    shader = '''
    void main() {
  float v = toNormalized(getDataValue(0)) * 15.0;
  float w = 1-v
  emitRGBA(vec4(w, w, w, v));
}
'''                                                 
    )
print(viewer) # Click the link that is generated to see your volume displayed in Neuroglancer

http://127.0.0.1:39235/v/aff71cd107c07e27ae2c855c4c57fee7e796ce72/


## Host the raw-space allen atlas cloudvolume

You will need to host this with cloudvolume as was done for the raw data above. Use a different port though, e.g.
```python
import cloudvolume
vol = cloudvolume.CloudVolume('file:///home/wanglab/Documents/neuroglancer/20170204_tp_bl6_cri_1750r_03/atlas')
vol.viewer(port=1338) # must be a different port than the raw data
```
Again, make sure to do this outside of this jupyter notebook

## Load in the atlas
Load in a segmentation layer (e.g. the raw-space allen atlas) that is being hosted with cloudvolume using the same viewer object that you already made. If you go back to the neuroglancer window after running this it should be there in a new layer

In [4]:
with viewer.txn() as s:
    s.layers['rawatlas'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    

Use the look-up table to make a key binding to the Neuroglancer session.

In [6]:
# First you need to create a dictionary mapping the atlas id to the region name. 
# Here's my example

csv_file = '/jukebox/LightSheetTransfer/atlas/ls_id_table_w_voxelcounts_16bit.xlsx'

df = pd.read_excel(csv_file)

atlas_df = df.copy()

atlas_dict = {}
for ind, row in atlas_df.iterrows():
    atlas_dict[int(row['id'])] = row['name']
    
# Here is the actual code for making the key binding. Copy and paste this
# into your jupyter notebook where you have already made the viewer() object

num_actions = 0
def my_action3(s):
    global num_actions
    num_actions += 1
    with viewer.config_state.txn() as st:
        region_id = s.selected_values['rawatlas']
        region_name = atlas_dict.get(region_id)
        st.status_messages['hello'] = ('%i:%s' %
                                    (region_id, region_name))

    print('Got my-action %i' % num_actions)
#     print('  Mouse position: %s' % (s.mouse_voxel_coordinates,))
    print('  Layer selected values: %s' % (s.selected_values,))
viewer.actions.add('my-action3', my_action3)

with viewer.config_state.txn() as s:
    s.input_event_bindings.viewer['keyp'] = 'my-action3'
    s.status_messages['hello'] = 'Welcome to the segment labeling example. Press "p" to see region name under cursor.'

Got my-action 1
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 2692, "rawatlas": 194})
Got my-action 2
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 2692, "rawatlas": 194})
Got my-action 3
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 738, "rawatlas": 327})
Got my-action 4
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 738, "rawatlas": 327})
Got my-action 5
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 1437, "rawatlas": 125})
Got my-action 6
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 1437, "rawatlas": 125})
Got my-action 7
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 4425, "rawatlas": 403})
Got my-action 8
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 4425, "rawatlas": 403})
Got my-action 9
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 804, "rawatlas": 1097})
Got my-action 10
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 804, 

Adjust the viewer and take a screenshot. Good for videos!

In [11]:
svdst = '/jukebox/wang/zahra/neuroglancer/screenshots/20170204_tp_bl6_cri_1750r_03/amygdala'
if not os.path.exists(svdst): os.mkdir(svdst)
for i in range(450,771):
    with viewer.txn() as s:
        s.voxel_coordinates = [1453,3260,i]
    ss = neuroglancer.ScreenshotSaver(viewer, svdst)
    ss.capture(index=i)   

Alternatively, can also take a single view screenshot of the current view.

In [6]:
svdst = '/home/wanglab/Documents/neuroglancer/screenshots/20170204_tp_bl6_cri_1750r_03/amygdala'
if not os.path.exists(svdst): os.mkdir(svdst)
ss = neuroglancer.ScreenshotSaver(viewer, svdst)
ss.capture(index=0) 

(0,
 '/home/wanglab/Documents/neuroglancer/screenshots/20170204_tp_bl6_cri_1750r_03/amygdala/0000000.png')

Got my-action 61
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 515, "rawatlas": 303})
Got my-action 62
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 618, "rawatlas": 334})
Got my-action 63
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 533, "rawatlas": 303})
Got my-action 64
  Layer selected values: Map({"20170204_tp_bl6_cri_1750r_03": 644, "rawatlas": 403})


Change the view layout to show the segmentation side by side with the image, rather than overlayed.  This can also be done from the UI by dragging and dropping.  The side by side views by default have synchronized position, orientation, and zoom level, but this can be changed.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01', 'overlay']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas'])])

Remove the overlay layer.

In [ ]:
with viewer.txn() as s:
    s.layout = neuroglancer.row_layout(
        [neuroglancer.LayerGroupViewer(layers=['20161205_tp_bl6_lob45_1000r_01']),
         neuroglancer.LayerGroupViewer(layers=['rawatlas'])])